In [1]:
import boto3
iam = boto3.client('iam')
response = iam.list_roles()
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/ravi.tej/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [3]:
import pandas as pd
import numpy as np
import json

import re
from transformers import AutoTokenizer
from random import randint
import sys
sys.path.append("../utils")
from pack_dataset import pack_dataset
from datasets import Dataset

import requests

In [4]:
tokenizer_model_id = 'teknium/OpenHermes-2.5-Mistral-7B'

In [5]:
# model_id = "TheBloke/OpenHermes-2.5-Mistral-7B-AWQ"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
all_responses = pd.read_csv('gpt-4-search-responses-5-feb.csv',on_bad_lines='skip',header=None)

In [7]:
all_responses

,0,1,2,3,4,5,6
0,0,4,social media influencers,16/02/24,"['65a7c2a01c9bc64bb8ae4a12', '65a7b6f11c9bc64b...",You're the lead editor for a financial and bus...,"I'm sorry, but the queries you've provided rel..."
1,0,5,kotak mahindra bank's banking and financial se...,23/12/23,"['65a662593c864051fa47ec9e', '65a9e5d56391ac19...",You're the lead editor for a financial and bus...,**Kotak Mahindra Bank Navigates Funding and Gr...
2,0,1,security token offerings news,05/02/24,"['65b320421b79671c6b70cca2', '65a91cba6391ac19...",You're the lead editor for a financial and bus...,**Exploring Recent Security Token Offerings an...
3,0,7,rock climbing basics,28/12/23,"['65bb79c3f3aa2a77a58bfe62', '65bb79cef3aa2a77...",You're the lead editor for a financial and bus...,**Understanding the Impact of Budget 2024 on I...
4,0,2,gold investment,01/01/24,"['65bb096df3aa2a77a58bfb1a', '65a896681c9bc64b...",You're the lead editor for a financial and bus...,**Gold Investment Outlook for 2024 Shows Promi...
...,...,...,...,...,...,...,...
2475,0,2638,strides pharma science's pharmaceuticals,23/12/23,"['658d3c054b25ce4814fb3002', '65be1cc3f3aa2a77...",You're the lead editor for a financial and bus...,**Strides Pharma Science Investment Outlook an...
2476,0,2636,united breweries' beer market share,10/02/24,"['6566e18fb878114e69597686', '6571c2a625cc9a34...",You're the lead editor for a financial and bus...,**United Breweries' Market Presence and Future...
2477,0,2630,assessment of india's retail sector in the cur...,29/01/24,"['65bb0953f3aa2a77a58bf901', '65b74d161b79671c...",You're the lead editor for a financial and bus...,**India's Retail Sector Assessment in Recent E...
2478,0,2633,e-learning resources,10/02/24,"['65bb0955f3aa2a77a58bf924', '65b89e8b1b79671c...",You're the lead editor for a financial and bus...,**Exploring Addictive Learning's Spectacular I...


In [8]:
delimiter = '|| user_query'

In [9]:
all_responses.iloc[5][5].split('|| user_query')

["You're the lead editor for a financial and business website. You answer user queries by using relevant articles that you've found by strictly following the below rules:\n*valid_queries: questions about finance, business, or relevant entities like India, RBI, Reliance, Fed etc\n*invalid_queries: non financial or business questions or inappropriate content\nresponse_instructions:\n1.query is valid and answerable with the articles you found - summarize the information clearly and concisely with a headline and followed by 3 to 6 points with following guidelines: \nPrefix the point with a relevant label one emoji for that point. The points should overall capture the essence of a good response for the user query, but at the same time as brief as possible - a few words to a maximum of 1 sentence per point. Start with the most important point for the query. The labels should be human readable avoid use of _ or programming symbols. Headline should be between 5 to 15 words and tailored to the 

In [10]:
def split_prompt(input_prompt):
    x, y = input_prompt.split('|| user_query')
    y = '|| user_query' + y
    return x, y

In [11]:
all_responses['system_prompt'], all_responses['user_prompt'] = zip(*all_responses[5].apply(lambda x: split_prompt(x)))

In [12]:
all_responses = all_responses.rename(columns= {6: 'response'})

In [13]:
all_responses.iloc[5]['system_prompt']

"You're the lead editor for a financial and business website. You answer user queries by using relevant articles that you've found by strictly following the below rules:\n*valid_queries: questions about finance, business, or relevant entities like India, RBI, Reliance, Fed etc\n*invalid_queries: non financial or business questions or inappropriate content\nresponse_instructions:\n1.query is valid and answerable with the articles you found - summarize the information clearly and concisely with a headline and followed by 3 to 6 points with following guidelines: \nPrefix the point with a relevant label one emoji for that point. The points should overall capture the essence of a good response for the user query, but at the same time as brief as possible - a few words to a maximum of 1 sentence per point. Start with the most important point for the query. The labels should be human readable avoid use of _ or programming symbols. Headline should be between 5 to 15 words and tailored to the q

In [14]:
all_responses

,0,1,2,3,4,5,response,system_prompt,user_prompt
0,0,4,social media influencers,16/02/24,"['65a7c2a01c9bc64bb8ae4a12', '65a7b6f11c9bc64b...",You're the lead editor for a financial and bus...,"I'm sorry, but the queries you've provided rel...",You're the lead editor for a financial and bus...,|| user_query: social media influencers || tod...
1,0,5,kotak mahindra bank's banking and financial se...,23/12/23,"['65a662593c864051fa47ec9e', '65a9e5d56391ac19...",You're the lead editor for a financial and bus...,**Kotak Mahindra Bank Navigates Funding and Gr...,You're the lead editor for a financial and bus...,|| user_query: kotak mahindra bank's banking a...
2,0,1,security token offerings news,05/02/24,"['65b320421b79671c6b70cca2', '65a91cba6391ac19...",You're the lead editor for a financial and bus...,**Exploring Recent Security Token Offerings an...,You're the lead editor for a financial and bus...,|| user_query: security token offerings news |...
3,0,7,rock climbing basics,28/12/23,"['65bb79c3f3aa2a77a58bfe62', '65bb79cef3aa2a77...",You're the lead editor for a financial and bus...,**Understanding the Impact of Budget 2024 on I...,You're the lead editor for a financial and bus...,|| user_query: rock climbing basics || today d...
4,0,2,gold investment,01/01/24,"['65bb096df3aa2a77a58bfb1a', '65a896681c9bc64b...",You're the lead editor for a financial and bus...,**Gold Investment Outlook for 2024 Shows Promi...,You're the lead editor for a financial and bus...,|| user_query: gold investment || today date: ...
...,...,...,...,...,...,...,...,...,...
2475,0,2638,strides pharma science's pharmaceuticals,23/12/23,"['658d3c054b25ce4814fb3002', '65be1cc3f3aa2a77...",You're the lead editor for a financial and bus...,**Strides Pharma Science Investment Outlook an...,You're the lead editor for a financial and bus...,|| user_query: strides pharma science's pharma...
2476,0,2636,united breweries' beer market share,10/02/24,"['6566e18fb878114e69597686', '6571c2a625cc9a34...",You're the lead editor for a financial and bus...,**United Breweries' Market Presence and Future...,You're the lead editor for a financial and bus...,|| user_query: united breweries' beer market s...
2477,0,2630,assessment of india's retail sector in the cur...,29/01/24,"['65bb0953f3aa2a77a58bf901', '65b74d161b79671c...",You're the lead editor for a financial and bus...,**India's Retail Sector Assessment in Recent E...,You're the lead editor for a financial and bus...,|| user_query: assessment of india's retail se...
2478,0,2633,e-learning resources,10/02/24,"['65bb0955f3aa2a77a58bf924', '65b89e8b1b79671c...",You're the lead editor for a financial and bus...,**Exploring Addictive Learning's Spectacular I...,You're the lead editor for a financial and bus...,|| user_query: e-learning resources || today d...


### Fixing inconsistent number of line breaks

In [15]:
import re

def convert_newlines_and_brs(text):
    # Define a pattern to match single or multiple \n, <br> tags with optional spaces between them
    # This pattern matches:
    # - one or more newline characters (\n+)
    # - or one or more sequences of <br> possibly surrounded by spaces (\s*<br>\s*)+
    pattern = r'(\n+|\s*<br>\s*)+'
    
    # Replace the matched patterns with <br><br>
    result = re.sub(pattern, '<br><br>', text)
    
    return result

In [16]:
all_responses['modified_response'] = all_responses['response'].apply(lambda x: convert_newlines_and_brs(x))

In [75]:
# all_responses['modified_response'] = all_responses['modified_response'].apply(lambda x: x.replace('<br><br> <br>','<br><br>'))
# all_responses['modified_response'] = all_responses['modified_response'].apply(lambda x: x.replace('<br><br><br>','<br><br>'))
# all_responses['modified_response'] = all_responses['modified_response'].apply(lambda x: x.replace('<br> <br><br>','<br><br>'))

### Changing our site to platform in response - changing the prompt as well

In [17]:
all_responses['has_our_site'] = all_responses['modified_response'].apply(lambda x: 'our site' in x.lower() or 'our website' in x.lower())

In [18]:
all_responses.groupby('has_our_site')['has_our_site'].count()

has_our_site
False    2421
True       59
Name: has_our_site, dtype: int64

In [19]:
all_responses['is_sorry'] = all_responses['modified_response'].apply(lambda x: 'sorry' in x.lower())

In [20]:
all_responses['modified_response'] = all_responses.apply(lambda x: x['modified_response'].replace('website','platform') if x['is_sorry'] else x['modified_response'],axis=1)
all_responses['modified_response'] = all_responses.apply(lambda x: x['modified_response'].replace('Website','Platform') if x['is_sorry'] else x['modified_response'],axis=1)
all_responses['modified_response'] = all_responses.apply(lambda x: x['modified_response'].replace('site','platform') if x['is_sorry'] else x['modified_response'],axis=1)
all_responses['modified_response'] = all_responses.apply(lambda x: x['modified_response'].replace('Site','Platform') if x['is_sorry'] else x['modified_response'],axis=1)

In [45]:
all_responses['starts_with_headline'] = all_responses['modified_response'].apply(lambda x: x[:2]=='**')

In [46]:
all_responses.groupby('starts_with_headline')['starts_with_headline'].count()

starts_with_headline
False     249
True     2231
Name: starts_with_headline, dtype: int64

In [51]:
all_responses['bad_response'] = all_responses['modified_response'].apply(lambda x: 'user_query' in x) 

In [53]:
all_responses.groupby('bad_response')['bad_response'].count()

bad_response
False    2478
True        2
Name: bad_response, dtype: int64

In [56]:
all_responses[(all_responses.starts_with_headline==False) & (all_responses.is_sorry==False)].loc[163].modified_response

"I apologize, but based on the articles provided, there is no specific information regarding Hindalco Industries' aluminum market strategies. The articles only cover recent stock price movements and trading data."

In [21]:
all_responses[all_responses.is_sorry == True].modified_response.values

array(["I'm sorry, but the queries you've provided relate to social media trends and influencer impact on consumer behavior, which are not directly tied to the finance and business focus areas covered on our platform. If you have any questions related to finance or business, I'd be glad to assist with those.",
       "I'm sorry, but your query about Taapsee Pannu thriller movies does not fall within the scope of finance or business topics that we cover on our platform. We focus on providing information and insights related to those areas. If you have any questions related to finance or business, feel free to ask, and I'd be happy to help!",
       'I\'m sorry, but the user query you provided is about a "fruit tart recipe," which is not related to finance or business topics. As the lead editor for a financial and business platform, I focus on providing information on financial and economic matters. If you have any questions about finance or business, feel free to ask, and I would be hap

In [22]:
all_responses['system_prompt'] = all_responses['system_prompt'].apply(lambda x: x.replace('website','platform'))

In [23]:
def format_as_prompt_response(response_row):
    messages = [{"role": "system", "content": response_row['system_prompt']},
                {"role": "user", "content": response_row['user_prompt']}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + response_row['modified_response']
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [28]:
all_responses = all_responses.rename(columns = {1: 'index',2:'query',3:'query_date',4:'article_ids',5: 'gpt_prompt'})

In [29]:
all_responses.head(10)

,0,index,query,query_date,article_ids,gpt_prompt,response,system_prompt,user_prompt,modified_response,has_our_site,is_sorry
0,0,4,social media influencers,16/02/24,"['65a7c2a01c9bc64bb8ae4a12', '65a7b6f11c9bc64b...",You're the lead editor for a financial and bus...,"I'm sorry, but the queries you've provided rel...",You're the lead editor for a financial and bus...,|| user_query: social media influencers || tod...,"I'm sorry, but the queries you've provided rel...",True,True
1,0,5,kotak mahindra bank's banking and financial se...,23/12/23,"['65a662593c864051fa47ec9e', '65a9e5d56391ac19...",You're the lead editor for a financial and bus...,**Kotak Mahindra Bank Navigates Funding and Gr...,You're the lead editor for a financial and bus...,|| user_query: kotak mahindra bank's banking a...,**Kotak Mahindra Bank Navigates Funding and Gr...,False,False
2,0,1,security token offerings news,05/02/24,"['65b320421b79671c6b70cca2', '65a91cba6391ac19...",You're the lead editor for a financial and bus...,**Exploring Recent Security Token Offerings an...,You're the lead editor for a financial and bus...,|| user_query: security token offerings news |...,**Exploring Recent Security Token Offerings an...,False,False
3,0,7,rock climbing basics,28/12/23,"['65bb79c3f3aa2a77a58bfe62', '65bb79cef3aa2a77...",You're the lead editor for a financial and bus...,**Understanding the Impact of Budget 2024 on I...,You're the lead editor for a financial and bus...,|| user_query: rock climbing basics || today d...,**Understanding the Impact of Budget 2024 on I...,False,False
4,0,2,gold investment,01/01/24,"['65bb096df3aa2a77a58bfb1a', '65a896681c9bc64b...",You're the lead editor for a financial and bus...,**Gold Investment Outlook for 2024 Shows Promi...,You're the lead editor for a financial and bus...,|| user_query: gold investment || today date: ...,**Gold Investment Outlook for 2024 Shows Promi...,False,False
5,0,0,regulatory frameworks,27/01/24,"['65b9b7d4f3aa2a77a58bf57f', '65b4aa001b79671c...",You're the lead editor for a financial and bus...,**Decoding Regulatory Frameworks: Data Protect...,You're the lead editor for a financial and bus...,|| user_query: regulatory frameworks || today ...,**Decoding Regulatory Frameworks: Data Protect...,False,False
6,0,9,bcci,18/01/24,"['65a894451c9bc64bb8ae4ac6', '659e08404b25ce48...",You're the lead editor for a financial and bus...,**BCCI Secures New Partnerships Amid Continued...,You're the lead editor for a financial and bus...,|| user_query: bcci || today date: 18 January ...,**BCCI Secures New Partnerships Amid Continued...,False,False
7,0,8,securities lending news today,25/01/24,"['65b23f561b79671c6b70cc68', '65b320421b79671c...",You're the lead editor for a financial and bus...,**Today's Update: Securities Lending News and ...,You're the lead editor for a financial and bus...,|| user_query: securities lending news today |...,**Today's Update: Securities Lending News and ...,False,False
8,0,6,sbi life insurance stock analysis,25/01/24,"['65b866521b79671c6b70d663', '65b9f00cf3aa2a77...",You're the lead editor for a financial and bus...,**Consistent Buy Ratings for SBI Life Insuranc...,You're the lead editor for a financial and bus...,|| user_query: sbi life insurance stock analys...,**Consistent Buy Ratings for SBI Life Insuranc...,False,False
9,0,3,pension fund returns today,14/02/24,"['65adda576391ac19baa871d6', '65bc5b59f3aa2a77...",You're the lead editor for a financial and bus...,**National Pension System Corpus Growth and Ta...,You're the lead editor for a financial and bus...,|| user_query: pension fund returns today || t...,**National Pension System Corpus Growth and Ta...,False,False


In [30]:
all_responses.to_csv('cleaned_gpt_responses-6-feb.csv',index=False)

In [31]:
all_responses.sample(20).modified_response.values

array(["**Understanding FPI Trends: 2023 Highlights and 2024 Outlook**<br><br>📈 **Record inflows**: FPIs bought Indian equities worth ₹1.65 lakh crore in 2023.<br><br>🔁 **Buying & Selling cycles**: Net sellers in January, February, September, and October; strong buyers between March-August and December.<br><br>🏦 **Debt market attraction**: In 2023, FPIs were net buyers in Indian debt for the first time in 4 years.<br><br>📊 **Debt vs Equity**: Notable ₹19,800 crore injected into Indian bonds in January 2024 amid equity withdrawals.<br><br>💹 **Equity market outlook**: Positive buying expected in 2024 due to anticipated rate cuts in the US.<br><br>🌐 **Global factors**: US bond yields impact and India’s inclusion in JP Morgan's index slated for mid-2024 could influence FPI trends. <br><br>Please note, it's imperative for investors to conduct their own rigorous research and consider consulting a financial expert before making investment decisions.",
       '**Global Concerns: Extreme Weathe

In [32]:
k = format_as_prompt_response(all_responses.iloc[10])

In [33]:
len(tokenizer.encode(k))

2357

In [34]:
from datasets import Dataset
sys.path.append("../utils")
from pack_dataset import pack_dataset
finetune_id = 'MangoPurpleValley'

In [35]:
train_df = all_responses[['system_prompt', 'user_prompt','modified_response']]
dataset = Dataset.from_pandas(train_df)

In [36]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_as_prompt_response(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset)
# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)
# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=4096) # We use 4096 as the maximum length for packing
print(f"Total number of samples: {len(lm_dataset)}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Total number of samples: 1926
Total number of samples: 1926


In [37]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [39]:
from datetime import datetime

In [42]:
len(train_df)

2480

In [40]:
finetune_dataset_config = {'finetune_id': 'search_openhermes_fp16_' + finetune_id,
                          'date': datetime.strftime(datetime.today(),'%Y-%m-%d'),
                          'num_datapoints': len(train_df),
                            'data_source': 'gpt4'}

In [41]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/1926 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-02-06-search_openhermes_fp16_MangoPurpleValley


### Capybara dataset

In [58]:
tokenizer = AutoTokenizer.from_pretrained('NousResearch/Nous-Capybara-34B',use_fast=True,trust_remote_code=True)

tokenization_yi.py:   0%|          | 0.00/8.96k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/NousResearch/Nous-Capybara-34B:
- tokenization_yi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

In [61]:
train_df = all_responses[['system_prompt', 'user_prompt','modified_response']]
dataset = Dataset.from_pandas(train_df)

In [60]:
def format_as_prompt_response(response_row):
    messages = [{"role": "system", "content": response_row['system_prompt']},
                {"role": "user", "content": response_row['user_prompt']}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + response_row['modified_response']
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [62]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_as_prompt_response(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset)
# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)
# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=4096) # We use 4096 as the maximum length for packing
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/2480 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using a default chat template that implements the ChatML format. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/2480 [00:00<?, ? examples/s]

Total number of samples: 1830
Total number of samples: 1830


In [63]:
finetune_dataset_config = {'finetune_id': 'search_capybara_fp16_' + finetune_id,
                          'date': datetime.strftime(datetime.today(),'%Y-%m-%d'),
                          'num_datapoints': len(train_df),
                            'data_source': 'gpt4'}

In [64]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/1830 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-02-10-search_capybara_fp16_MangoPurpleValley
